In [1]:
'''
В данной версии КП будут использоваться модели, предложенные наставником, а также будет проведено
исследование влияния разделения признаков на результат
'''

import numpy as np, pandas as pd, os, seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RFReg


wrk='C:\\Users\\Lenovo\\Documents\\GeekBrains\\Python DataScience Kurs'
train=pd.read_csv(os.path.join(wrk,'train.csv'))

train=train.drop(['Id'],axis=1) #удаляем стоблбец id - он нам не нужен
cols=train.columns.to_numpy()

res={}

for x in cols:
    res[x]=train[x].value_counts().sort_index()

In [2]:
'''
Результаты анализа значений:
    
    DistrictId - 205 возможных значений. Прямой/обратной зависимости (чем больше id, тем лучше/хуже район)
            стоимости квартиры от данного параметра не предполагается. Поэтому отнесем к дискретным;
    Rooms - 9 различных значений. Предполагается зависимость цены квартиры от кол-ва комнат, поэтому отнесем этот
            параметр к непрерывным;
    Square - 10000 различных значений. ОТносим к непрерывным признакам;
    LifeSquare - 7887 различных значений. Относим к непрерывным значениям; Значения nan будем заменять значениями
            Square-KitchenSquare ПОСЛЕ редактирования KitchenSquare;
    KitchenSquare - 58 различных значений. Предполагается зависимость цены квартиры от площади кухни, поэтому отнесем этот
            параметр к непрерывным. Почти 25% значений равны 1 => заменим их на медиану без учета 1.
            Есть ошибочные значения 1970 и 2014 (видимо, год постройки дома);
    Floor - 33 различных значения. Предполагается зависимость цены квартиры от ее этажа, поэтому отнесем этот
            параметр к непрерывным;
    HouseFloor - 44 различных значения. Предполагается зависимость цены квартиры от этажности дома, поэтому
            отнесем этот параметр к непрерывным; ОБРАБОТКА ОШИБОЧНЫХ ЗНАЧЕНИЙ: Floor <= HouseFloor и HouseFloor!=0;
    HouseYear - 97 различных значений. Предполагается зависимость цены квартиры от года постройки дома, поэтому
            отнесем этот параметр к непрерывным; Ошибочные значения: 4968 и 20052011;
    Ecology_1 - 129 непрерывных значений. Отнесем этот параметр к непрерывным;
    Ecology_2 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    Ecology_3 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    Social_1 - 51 различное значение. Предполагается зависимость цены квартиры от социальных условий, поэтому
            отнесем этот параметр к непрерывным;
    Social_2 - 142 различных значения. Аналогично;
    Social_3 - 30 различных значений. Аналогично;
    Healthcare_1 - 79 различных значений. Аналогично;
    Helthcare_2 - 7 различных значений. Из-за этого отнесем этот параметр к дискретным;
    Shops_1 - 16 различных значений. Предполагается зависимость цены квартиры от доступности магазинов, поэтому
            отнесем этот параметр к непрерывным;
    Shops_2 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    
'''

#заполняем nan-значения предыдущими
train=train.fillna(method='pad')
train['Healthcare_1'][0]=train['Healthcare_1'].median() #в первой строке осталось nan-значение

#переводим категориальные переменные в dummy-переменные
train=pd.get_dummies(train,prefix=['eco2','eco3','sh2'])
cols=train.columns.to_numpy()
cols_x=list(cols)
cols_x.remove('Price')

'''определление важности признаков'''
feats_tab=train.copy()
answ_tab=feats_tab['Price']
feats_tab=feats_tab.drop(['Price'],axis=1)

feats_tab=feats_tab.to_numpy()
answ_tab=answ_tab.to_numpy()

<ipython-input-2-a24db798ef3c>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Healthcare_1'][0]=train['Healthcare_1'].median() #в первой строке осталось nan-значение


In [3]:
'''
Обработку данных будем проводить как во второй версии КП.
Будем рассматривать 2 модели случайного леса с одинаковыми параметрами (за исключением максимального кол-ва
признаков). В первом случае будем рассматривать все признаки, а во втором - 7
'''

#подготовка тестовой выборки к разделению для обучения множества регрессоров
X=train.copy()
y=X['Price']
X=X.drop(['Price'],axis=1)
X=X.to_numpy()
y=y.to_numpy()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)

#построение моделей
model1 = RFReg(n_estimators=1442, max_depth=18, random_state=42) #весь 21 признак
model1.fit(X_train,y_train)
test_react=model1.predict(X_test)
r2_mdl1=r2_score(y_test,test_react)
print('Значение r2 первой модели: '+str(r2_mdl1))

model2 = RFReg(n_estimators=1442, max_depth=18, random_state=42, max_features=7) # 7 признаков
model2.fit(X_train,y_train)
test_react=model2.predict(X_test)
r2_mdl2=r2_score(y_test,test_react)
print('Значение r2 второй модели: '+str(r2_mdl2))

Значение r2 первой модели: 0.7204341123674933
Значение r2 второй модели: 0.7360265270645087


In [ ]:
'''
Наибольшее значение r2 показал ансамбль с максимальным кол-вом признаков, равным 7 (0.7360).
Следовательно, разделение признаков положительно влияет на процесс обучения.
Обучим вторую модель на тестовой выборке
'''

reg=RFReg(n_estimators=1442, max_depth=18, random_state=42, max_features=7)
reg.fit(feats_tab,answ_tab)

#проверка модели
test=pd.read_csv(os.path.join(wrk,'test.csv'))
test=test.drop(['Id'],axis=1) #удаляем стоблбец id - он нам не нужен

'''В тестовых данных также имеются пропуски и неточности. Обработаем их точно так же, как и обучающие данные:'''
test=test.fillna(method='pad')
test=pd.get_dummies(test,prefix=['eco2','eco3','sh2'])
test['Healthcare_1'][0:2]=test['Healthcare_1'].median() #в первой строке осталось nan-значение

#загружаем тестовые данные в модель и вычисляем результат
test_pred=reg.predict(test.to_numpy())
test_pred=pd.Series(test_pred)

test_pred_to_save=pd.read_csv(os.path.join(wrk,'sample_submission.csv')) #шаблон для загрузки на Kaggle
test_pred_to_save['Price']=test_pred
test_pred_to_save.to_csv(os.path.join(wrk,'test_pred.csv'),index=False)